## Drop-OFF Analysis

In [ ]:
import pandas as pd

### Data Loading & Overview

In [113]:
pageviews = pd.read_csv(r"D:\Data Analysis Learning\Portfolio Projects\toy-store-ecommerce-analysis\Toy Store E-Commerce Database\website_pageviews.csv")

In [14]:
pageviews.sort_values('website_session_id').head(100)

,website_pageview_id,created_at,website_session_id,pageview_url
0,1,2012-03-19 08:04:16,1,/home
1,2,2012-03-19 08:16:49,2,/home
2,3,2012-03-19 08:26:55,3,/home
3,4,2012-03-19 08:37:33,4,/home
4,5,2012-03-19 09:00:55,5,/home
...,...,...,...,...
96,97,2012-03-19 14:14:07,54,/products
98,99,2012-03-19 14:18:16,54,/the-original-mr-fuzzy
95,96,2012-03-19 14:12:09,54,/home
97,98,2012-03-19 14:15:08,55,/home


### Drop-off Rate Calculation

#### Where do customers drop off in the purchase funnel?

In [ ]:

last_page = pd.DataFrame(pageviews.groupby('website_session_id')['pageview_url'].last())

In [ ]:
drop_off = pd.DataFrame(last_page['pageview_url'].value_counts())
drop_off['pageview_url'] = drop_off.index

In [91]:
drop_off.index = range(len(drop_off.index))

In [121]:
drop_off = drop_off[['pageview_url', 'count']]


In [122]:
drop_off.rename(columns={'count' : 'drop_off_count'})

,pageview_url,drop_off_count
0,/the-original-mr-fuzzy,92568
1,/lander-2,59249
2,/home,57346
3,/products,51017
4,/lander-3,39733
5,/thank-you-for-your-order,32313
6,/cart,30469
7,/lander-1,25330
8,/lander-5,25131
9,/billing-2,17748


#### How many times each page in the funnel is visited?

In [ ]:
page_enterance = pd.DataFrame(pageviews.groupby('pageview_url')['website_session_id'].count())
page_enterance['pageview_url'] = page_enterance.index

In [86]:
page_enterance.index = range(len(page_enterance.index))
page_enterance = page_enterance[['pageview_url','website_session_id']]

In [ ]:
page_enterance

,pageview_url,website_session_id
0,/billing,3617
1,/billing-2,48441
2,/cart,94953
3,/home,137576
4,/lander-1,47574
5,/lander-2,131170
6,/lander-3,79000
7,/lander-4,9385
8,/lander-5,68166
9,/products,261231


#### Mergin optained results to a single dataframe

In [99]:
drop_off_rate = pd.merge(page_enterance, drop_off, left_on='pageview_url', right_on='pageview_url', how='left')
drop_off_rate.columns = ['pageview_url','page_enterance_count', 'drop_off_count']
drop_off_rate

,pageview_url,page_enterance_count,drop_off_count
0,/billing,3617,1997
1,/billing-2,48441,17748
2,/cart,94953,30469
3,/home,137576,57346
4,/lander-1,47574,25330
5,/lander-2,131170,59249
6,/lander-3,79000,39733
7,/lander-4,9385,4851
8,/lander-5,68166,25131
9,/products,261231,51017


#### Calculating Drop-off Rates on page visits basis and on total sessions basis

In [109]:
drop_off_rate['drop_off_rate'] = drop_off_rate['drop_off_count'] * 100 / drop_off_rate['page_enterance_count']
drop_off_rate['drop_off_rate_of_total'] = drop_off_rate['drop_off_count'] * 100 / drop_off_rate['drop_off_count'].sum()
drop_off_rate[['drop_off_rate','drop_off_rate_of_total']] = round(drop_off_rate[['drop_off_rate','drop_off_rate_of_total']], 2)


In [112]:
drop_off_rate.sort_values(by='drop_off_rate_of_total', ascending=False)

,pageview_url,page_enterance_count,drop_off_count,drop_off_rate,drop_off_rate_of_total
15,/the-original-mr-fuzzy,162525,92568,56.96,19.58
5,/lander-2,131170,59249,45.17,12.53
3,/home,137576,57346,41.68,12.13
9,/products,261231,51017,19.53,10.79
6,/lander-3,79000,39733,50.29,8.40
11,/thank-you-for-your-order,32313,32313,100.00,6.83
2,/cart,94953,30469,32.09,6.44
4,/lander-1,47574,25330,53.24,5.36
8,/lander-5,68166,25131,36.87,5.31
1,/billing-2,48441,17748,36.64,3.75


## Behavior Comparison

### Converted 🆚 non-converted sessions to orders

In [127]:
orders = pd.read_csv(r"d:\Data Analysis Learning\Portfolio Projects\toy-store-ecommerce-analysis\Toy Store E-Commerce Database\orders.csv")

sessions = pd.read_csv(r"d:\Data Analysis Learning\Portfolio Projects\toy-store-ecommerce-analysis\Toy Store E-Commerce Database\website_sessions.csv")

#### converted_sessions table construction

In [157]:
# converted_sessions = orders[['website_session_id', 'order_id', 'created_at']]
converted_sessions= pd.merge(orders, sessions, left_on='website_session_id', right_on='website_session_id', how='left',suffixes=("_o","_w"))
converted_sessions = converted_sessions[['website_session_id', 'user_id_w', 'created_at_w', 'is_repeat_session', 'utm_source', 'utm_campaign','utm_content', 'device_type']]

In [158]:
converted_sessions = pd.merge(converted_sessions, pageviews, left_on='website_session_id', right_on='website_session_id', how='left',suffixes=("_c","_p"))

In [159]:
converted_sessions

,website_session_id,user_id_w,created_at_w,is_repeat_session,utm_source,utm_campaign,utm_content,device_type,website_pageview_id,created_at,pageview_url
0,20,20,2012-03-19 10:22:58,0,gsearch,nonbrand,g_ad_1,desktop,30,2012-03-19 10:22:58,/home
1,20,20,2012-03-19 10:22:58,0,gsearch,nonbrand,g_ad_1,desktop,32,2012-03-19 10:28:09,/products
2,20,20,2012-03-19 10:22:58,0,gsearch,nonbrand,g_ad_1,desktop,33,2012-03-19 10:28:37,/the-original-mr-fuzzy
3,20,20,2012-03-19 10:22:58,0,gsearch,nonbrand,g_ad_1,desktop,34,2012-03-19 10:31:29,/cart
4,20,20,2012-03-19 10:22:58,0,gsearch,nonbrand,g_ad_1,desktop,35,2012-03-19 10:34:43,/shipping
...,...,...,...,...,...,...,...,...,...,...,...
226186,472818,386000,2015-03-19 05:26:56,1,NaN,NaN,NaN,desktop,1187990,2015-03-19 05:28:43,/the-original-mr-fuzzy
226187,472818,386000,2015-03-19 05:26:56,1,NaN,NaN,NaN,desktop,1187993,2015-03-19 05:31:28,/cart
226188,472818,386000,2015-03-19 05:26:56,1,NaN,NaN,NaN,desktop,1187995,2015-03-19 05:34:34,/shipping
226189,472818,386000,2015-03-19 05:26:56,1,NaN,NaN,NaN,desktop,1187997,2015-03-19 05:36:59,/billing-2


#### non_converted_sessions table construction

In [151]:
non_converted_sessions= pd.merge(orders, sessions, left_on='website_session_id', right_on='website_session_id', how='right_anti',suffixes=("_n","_w"))
non_converted_sessions = non_converted_sessions[['website_session_id','user_id_w','created_at_w', 'is_repeat_session', 'utm_source', 'utm_campaign','utm_content', 'device_type']]

In [152]:
non_converted_sessions = pd.merge(non_converted_sessions, pageviews, left_on='website_session_id', right_on='website_session_id', how='left',suffixes=("_n","_p"))

In [153]:
non_converted_sessions

,website_session_id,user_id_w,created_at_w,is_repeat_session,utm_source,utm_campaign,utm_content,device_type,website_pageview_id,created_at,pageview_url
0,1,1,2012-03-19 08:04:16,0,gsearch,nonbrand,g_ad_1,mobile,1,2012-03-19 08:04:16,/home
1,2,2,2012-03-19 08:16:49,0,gsearch,nonbrand,g_ad_1,desktop,2,2012-03-19 08:16:49,/home
2,3,3,2012-03-19 08:26:55,0,gsearch,nonbrand,g_ad_1,desktop,3,2012-03-19 08:26:55,/home
3,4,4,2012-03-19 08:37:33,0,gsearch,nonbrand,g_ad_1,desktop,4,2012-03-19 08:37:33,/home
4,5,5,2012-03-19 09:00:55,0,gsearch,nonbrand,g_ad_1,mobile,5,2012-03-19 09:00:55,/home
...,...,...,...,...,...,...,...,...,...,...,...
961928,472869,394316,2015-03-19 07:55:40,0,gsearch,nonbrand,g_ad_1,mobile,1188116,2015-03-19 07:55:40,/lander-3
961929,472870,394317,2015-03-19 07:56:29,0,gsearch,nonbrand,g_ad_1,desktop,1188118,2015-03-19 07:56:29,/lander-5
961930,472870,394317,2015-03-19 07:56:29,0,gsearch,nonbrand,g_ad_1,desktop,1188119,2015-03-19 07:57:22,/products
961931,472870,394317,2015-03-19 07:56:29,0,gsearch,nonbrand,g_ad_1,desktop,1188121,2015-03-19 07:58:13,/the-original-mr-fuzzy


### Mobile 🆚 desktop